In [18]:
%config IPCompleter.greedy=True
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from datetime import datetime

In [19]:
url = 'https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares'
response = requests.get(url=url)
soup = BeautifulSoup(response.text, 'lxml')

In [20]:
table = soup.find('table', {'class': 'table table-striped table-responsive-md'})
df = pd.read_html(str(table))
df = df[0]
df.columns = ['rank', 'x_classification', 'date', 'region', 'start_time', 'maximum_time', 'end_time', 'movie']
df.head()

,rank,x_classification,date,region,start_time,maximum_time,end_time,movie
0,1,X28.0,2003/11/04,486,19:29,19:53,20:06,MovieView archive
1,2,X20.0,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2,2003/10/28,486,09:51,11:10,11:24,MovieView archive
3,4,X17.0,2005/09/07,808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


In [21]:
df = df.drop('movie', axis=1)
df.head()

,rank,x_classification,date,region,start_time,maximum_time,end_time
0,1,X28.0,2003/11/04,486,19:29,19:53,20:06
1,2,X20.0,2001/04/02,9393,21:32,21:51,22:03
2,3,X17.2,2003/10/28,486,09:51,11:10,11:24
3,4,X17.0,2005/09/07,808,17:17,17:40,18:03
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55


In [22]:
for index, row in df.iterrows():
    print(row)
    print()

rank                         1
x_classification         X28.0
date                2003/11/04
region                     486
start_time               19:29
maximum_time             19:53
end_time                 20:06
Name: 0, dtype: object

rank                         2
x_classification         X20.0
date                2001/04/02
region                    9393
start_time               21:32
maximum_time             21:51
end_time                 22:03
Name: 1, dtype: object

rank                         3
x_classification         X17.2
date                2003/10/28
region                     486
start_time               09:51
maximum_time             11:10
end_time                 11:24
Name: 2, dtype: object

rank                         4
x_classification         X17.0
date                2005/09/07
region                     808
start_time               17:17
maximum_time             17:40
end_time                 18:03
Name: 3, dtype: object

rank                         5
x_cla